## Source Code 
#### Grid World Enviroment setting
* states, actions, transition probability

In [31]:
# set state
import numpy as np
nCols = 3
nRows = 4
nWalls = 1
states = []
for i in range(nCols*nRows-nWalls):
    states.append(i)
N_STATES = len(states)
#print(N_STATES)
#print(states)

# set map
map = -np.ones((nCols+2,nRows+2))
for i in range(nCols):
    for j in range(nRows):
        map[i+1,j+1] = 0
map[2,2] = -1 # add wall
#print(map)

# set action
actions = [0, 1, 2, 3]
N_ACTIONS = len(actions)

# states -> location
locations = []
index = 0
for i in range(nCols):
    for j in range(nRows):
        if map[i+1,j+1]==0:
            locations.append((i+1,j+1))
            index = index + 1
#print(locations) # match index with states
# action -> move
move = [(0,-1),(-1,0),(0,1),(1,0)] # match index with actions
#print(move)

# set transition probability
P = np.zeros((N_STATES,N_ACTIONS,N_STATES)) # P[S,A,S']
for s in range(N_STATES):
    for a in range(N_ACTIONS):
        current_location = locations[s]
        # heading collectly  ####################################################################################
        next_location = (current_location[0] + move[a][0],current_location[1] + move[a][1])
        
        if map[next_location[0],next_location[1]] == -1: # there is barrier or wall
            next_location = current_location
            next_s = states[locations.index(next_location)]
        else:
            next_s = states[locations.index(next_location)]
        P[s,a,next_s] = P[s,a,next_s] + 0.8
        # left error ############################################################################################
        next_location = (current_location[0] + move[a-1][0],current_location[1] + move[a-1][1])
        if map[next_location[0],next_location[1]] == -1: # there is barrier or wall
            next_location = current_location
            next_s = states[locations.index(next_location)]
        else:
            next_s = states[locations.index(next_location)]
        P[s,a,next_s] = P[s,a,next_s] + 0.1
        # right error ############################################################################################
        next_location = (current_location[0] + move[(a+1)%4][0],current_location[1] + move[(a+1)%4][1])
        
        if map[next_location[0],next_location[1]] == -1: # there is barrier or wall
            next_location = current_location
            next_s = states[locations.index(next_location)]
        else:
            next_s = states[locations.index(next_location)]
        P[s,a,next_s] = P[s,a,next_s] + 0.1
        
# rewards s,a ---  R(s,a)  ---> s'
if True:
    R = -0.02*np.ones((N_STATES,N_ACTIONS))
else:
    R = -0.5*np.ones((N_STATES,N_ACTIONS))
R[3,:] = 1
R[6,:] = -1
#print(R)
# discount factor
gamma = 0.99

# policy : given state which action would u choose
# assume that we know the policy
bad_policy = np.zeros((N_STATES,N_ACTIONS))
bad_policy[0,2] = 1
bad_policy[1,2] = 1
bad_policy[2,2] = 1
bad_policy[3,2] = 1
bad_policy[4,3] = 1
bad_policy[5,2] = 1
bad_policy[6,2] = 1
bad_policy[7,2] = 1
bad_policy[8,2] = 1
bad_policy[9,2] = 1
bad_policy[10,1] = 1

random_policy = 0.25*np.ones((N_STATES,N_ACTIONS))

optimal_policy = np.zeros((N_STATES,N_ACTIONS))
optimal_policy[0,2] = 1
optimal_policy[1,2] = 1
optimal_policy[2,2] = 1
optimal_policy[3,2] = 1
optimal_policy[4,1] = 1
optimal_policy[5,1] = 1
optimal_policy[6,1] = 1
optimal_policy[7,1] = 1
optimal_policy[8,0] = 1
optimal_policy[9,0] = 1
optimal_policy[10,0] = 1
#print(optimal_policy)

optimalWithNoise_policy = np.zeros((N_STATES,N_ACTIONS))
ep = 0.1
optimalWithNoise_policy[0,2] = 1
optimalWithNoise_policy[1,2] = 1
optimalWithNoise_policy[2,2] = 1
optimalWithNoise_policy[3,2] = 1
optimalWithNoise_policy[4,1] = 1
optimalWithNoise_policy[5,1] = 1
optimalWithNoise_policy[6,1] = 1
optimalWithNoise_policy[7,1] = 1
optimalWithNoise_policy[8,0] = 1
optimalWithNoise_policy[9,0] = 1
optimalWithNoise_policy[10,0] = 1
optimalWithNoise_policy = optimalWithNoise_policy + (ep/4)*np.ones((N_STATES,N_ACTIONS))
optimalWithNoise_policy = optimalWithNoise_policy / np.sum(optimalWithNoise_policy,axis = 1).reshape((N_STATES,1))

## SARSA and SARSA lambda

#### SARSA
* It is similar to TD(0)
* Main difference is TD(0) uses to evaluate policy,but SARSA evaulates policy and improves policy using epsilon greed method to gurantee continual exploraion
$$\:\:$$
$$Q(s_{t},a_{t})\: \leftarrow\:Q(s_{t},a_{t})\:+\:\alpha\:*\:(R_{t+1}\:+\:\gamma Q(s_{t+1},a_{t+1})-Q(s_{t},a_{t}))$$
$$\:\:$$
$$ \pi(a|s) = \begin{cases} 
     \epsilon /m +1-\epsilon\:\:\:\:if \:\:a^{*}=arg\max_{a \in A}Q(s,a) \\
     \epsilon /m \:\:\:\: otherwise\\
   \end{cases}
$$

#### SARSA lambda

* It is similar to TD($\lambda$)
$$Q(s,a) \leftarrow Q(s,a)+ \alpha \delta_{t} E_{t}(s,a) $$  

$$\:\:$$
$$ where,\:\: \delta_{t} = R_{t+1}\:+\:\gamma Q(S_{t+1},A_{t})-Q(S_{t},A_{t})$$
$$\:\:$$
$$ E_{t}(s,a) = \gamma \lambda E_{t-1}(s,a)+1(S=S_{t},A=A_{t})$$


In [32]:
## SARSA

def Egreedy(eps,sQ):
    prob = np.random.uniform(0,1)
    if prob>0.1:
        return np.argmax(sQ)
    else:
        return np.random.choice(actions)
    
## set Hyper parameters
epoch = 10000
alpha = 0.01
eps = 0.1
## set boundary condition
Q = np.zeros((N_STATES,N_ACTIONS))
Q[3,:] = 1.0; #goal
Q[6,:] = -1.0; #fail
## states
terminal_states =[3,6]
start_states = [x for x in states if x not in terminal_states]

for _ in range(epoch):
    done = False
    s = np.random.choice(start_states) # random initial state
    a = Egreedy(eps,Q[s,:]) # random initial action
    while not done:
        # s,a,r,s',a'
        reward = R[s,a]
        s1 = np.random.choice(states,p=P[s,a,:])
        # choose action followng epslion greedy
        a1 = Egreedy(eps,Q[s1,:])
        TD_target = reward + gamma * Q[s1,a1]
        Q[s,a] += alpha*(TD_target - Q[s,a])
        if (s1==3) or (s1==6):
            done = True
        else:
            s = s1
            a = a1
        
policy = np.eye(N_ACTIONS)[np.argmax(Q,axis=1)].astype(np.int32)        
print(Q)
print()
print(policy)

[[ 0.38263631  0.3710673   0.77966261  0.32469697]
 [ 0.47526795  0.55321764  0.85965791  0.55087806]
 [ 0.73657767  0.82285458  0.93453962  0.59723866]
 [ 1.          1.          1.          1.        ]
 [ 0.21422818  0.71098731  0.18697073  0.13608655]
 [ 0.47308883  0.67180262 -0.5867729   0.25589752]
 [-1.         -1.         -1.         -1.        ]
 [ 0.08376768  0.12610414  0.4340622   0.07886718]
 [ 0.19510033  0.23732698  0.47069782  0.19716453]
 [ 0.31521713  0.52385982  0.22850195  0.33963809]
 [ 0.26934153 -0.30306648  0.00204687  0.08277133]]

[[0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]]


In [33]:
## SARSA lambda

def Egreedy(eps,sQ):
    prob = np.random.uniform(0,1)
    if prob>0.1:
        return np.argmax(sQ)
    else:
        return np.random.choice(actions)
    
## set Hyper parameters
epoch = 5000
alpha = 0.01
lam = 0.5
eps = 0.1

## set boundary condition
Q = np.zeros((N_STATES,N_ACTIONS))
Q[3,:] = 1.0; #goal
Q[6,:] = -1.0; #fail

## states
terminal_states =[3,6]
start_states = [x for x in states if x not in terminal_states]

for _ in range(epoch):
    done = False
    
    #set Eligibilty traces
    E = np.zeros((N_STATES,N_ACTIONS))
    
    s = np.random.choice(start_states) # random initial state
    a = Egreedy(eps,Q[s,:]) # random initial action
    while not done:
        # s,a,r,s',a'
        reward = R[s,a]
        s1 = np.random.choice(states,p=P[s,a,:])
        a1 = Egreedy(eps,Q[s1,:])

        TD_target = reward + gamma * Q[s1,a1]
        TD_error = TD_target - Q[s,a]
        E[s,a]+=1
        
        for state in start_states:
            for action in actions:
                Q[state,action] += alpha*TD_error*E[state,action]
                E[state,action] = gamma*lam*E[state,action]
                
        if (s1==3) or (s1==6):
            done = True
        else:
            s = s1
            a = a1
           
        
policy = np.eye(N_ACTIONS)[np.argmax(Q,axis=1)].astype(np.int32)      
print(Q)
print()
print(policy)

[[ 0.25695284  0.35445463  0.7725692   0.31705707]
 [ 0.42963697  0.39385691  0.83475783  0.42267553]
 [ 0.64885198  0.64447305  0.88459019  0.47260521]
 [ 1.          1.          1.          1.        ]
 [ 0.16638398  0.72157964  0.1985347   0.1666512 ]
 [ 0.44372762  0.63058826 -0.58626212  0.22558002]
 [-1.         -1.         -1.         -1.        ]
 [ 0.09930968  0.64296816  0.058027    0.05700974]
 [ 0.08653609  0.05192211  0.48246976  0.11806629]
 [ 0.20431932  0.55392741  0.09877312  0.17293692]
 [ 0.29634224 -0.21079455 -0.00245975  0.03839282]]

[[0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]]


## Q-Learning

$$Q(s,a)\: \leftarrow\:Q(s,a)\:+\:\alpha\:*\:(R_{t+1}\:+\:\gamma \max_{s'}Q(s',a')-Q(s,a))$$

In [34]:
## Q-learning
def Egreedy(eps,sQ):
    prob = np.random.uniform(0,1)
    if prob>0.1:
        return np.argmax(sQ)
    else:
        return np.random.choice(actions)
    
## set Hyper parameters
epoch = 5000
alpha = 0.01
lam = 0.5
eps = 0.1

## set boundary condition
Q = np.zeros((N_STATES,N_ACTIONS))
Q[3,:] = 1.0; #goal
Q[6,:] = -1.0; #fail

## states
terminal_states =[3,6]
start_states = [x for x in states if x not in terminal_states]

for _ in range(epoch):
    done = False
    s = np.random.choice(start_states)
    a = Egreedy(eps,Q[s,:])
    while not done:
        # sa r s'a'
        reward = R[s,a]
        
        s1 = np.random.choice(states,p=P[s,a,:])
        a1 = Egreedy(eps,Q[s1,:])
        Q[s,a] += alpha*(reward + gamma*np.max(Q[s1,:])-Q[s,a])
        if s1 in terminal_states:
            done = True
        else:
            s = s1
            a = a1

            
print(Q)
print()
policy = np.eye(N_ACTIONS)[np.argmax(Q,axis=1)].astype(np.int32)
print(policy)

[[ 0.21133308  0.23828581  0.84744808  0.21029083]
 [ 0.30940754  0.37487895  0.8901476   0.32701586]
 [ 0.62743622  0.69990103  0.92917226  0.50942265]
 [ 1.          1.          1.          1.        ]
 [ 0.14078738  0.81103296  0.12507177  0.09246588]
 [ 0.41436474  0.63782759 -0.38882892  0.2163459 ]
 [-1.         -1.         -1.         -1.        ]
 [ 0.04045299  0.08988973  0.5587349   0.06927967]
 [ 0.13088359  0.15071278  0.58335587  0.14030293]
 [ 0.22624291  0.61157514  0.14416789  0.22493183]
 [ 0.46043638 -0.26227035  0.01431345  0.02346715]]

[[0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 1 0 0]
 [1 0 0 0]]


## Discussion
* Due to small sample size(batch size), 3 algorithms (SARSA,SARSA_lambda,Q-learning) have high variance in their output.